In [1]:
import pandas as pd
import time

time_start=time.time()
answer_info=pd.read_csv('answer_info.txt',sep='\s+',names=['aid','qid','uid','atime','ainfo','ainfoc','fine',
                                                                 'recommend','table','picture','video','numword',
                                                                 'thumbsup','thumbsdown','comment','collect','thank',
                                                                 'report','nohelp','oppose'])
ques_info=pd.read_csv('question_info.txt',sep='\s+',names=['qid','qtime','qtitle','qtitlec','qinfo','qinfoc','qtopic'])

In [2]:
ques_info=ques_info[['qid','qtopic']]

In [3]:
member_info=pd.read_csv('member_info.txt',sep='\s+',names=['uid','sex','key_word','num_level','hot_level','regis_type','regis_platform',
                                                          'look_freq','a','b','c','d','e','A','B','C','D','E','salt','l_topic','topic_n'])
member_info=member_info[['uid','sex','look_freq','a','b','c','d','e','A','B','C','D','E','salt']]

In [ ]:
##########################预处理

In [ ]:
from tqdm import tqdm
def flatten_topic(df):    
    a=[]
    for i in tqdm(range(len(df['qtopic'].values))):
        a += df['qtopic'].values[i].split(',')    
    new_df = pd.DataFrame()
    new_df['qtopic'] = a
    return new_df

In [5]:
topic_df = flatten_topic(ques_info)

100%|██████████| 1829900/1829900 [00:11<00:00, 158127.79it/s]


In [6]:
topic_df=topic_df.drop_duplicates(subset=['qtopic'])

In [7]:
topic_df=topic_df.reset_index(drop=True)

In [8]:
from sklearn.preprocessing import LabelEncoder
lbl = LabelEncoder()
lbl.fit(topic_df['qtopic'].astype('str'))
import pickle
with open('meta_qtopic.pickle', 'wb') as f:
      pickle.dump(lbl, f)

In [9]:
# with open('meta_qtopic.pickle', 'rb') as f:
#       lbl = pickle.load(f)

In [13]:
cat_col=['sex','look_freq','A','B','C','D','E']
import pickle
from tqdm import tqdm
from sklearn.preprocessing import LabelEncoder
for feat in tqdm(cat_col):
    from sklearn.preprocessing import LabelEncoder
    lbl = LabelEncoder()
    lbl.fit(member_info[feat].astype('str'))
    with open('meta_'+str(feat)+'.pickle', 'wb') as f:
        pickle.dump(lbl, f)

100%|██████████| 7/7 [00:01<00:00,  4.83it/s]


In [ ]:
####################################################

In [4]:
invite_info=pd.read_csv('invite_info.txt',sep='\s+',names=['qid','uid','time','target'])
invite_info=pd.merge(invite_info,ques_info,on='qid',how='left')

In [5]:
invite_info=invite_info.dropna(subset=['qtopic'])
invite_info['qtopic'] = invite_info['qtopic'].map(lambda x:x.split(','))
invite_info['qtopic_len'] = invite_info['qtopic'].map(lambda x:len(x))

In [6]:
from tqdm import tqdm

def flatten_topic(df):    
    u = []
    a = []
    q=[]
    for i in tqdm(range(len(df['qtopic'].values))):
        u += [df['uid'].values[i]]*df['qtopic_len'].values[i]
        q += [df['qid'].values[i]]*df['qtopic_len'].values[i]
        a += list(df['qtopic'].values[i])
        
    new_df = pd.DataFrame()
    new_df['uid'] = u
    new_df['qid'] = q
    new_df['qtopic'] = a
        
    return new_df

In [7]:
deal_invite_info = flatten_topic(invite_info)

100%|██████████| 9489162/9489162 [03:04<00:00, 51480.35it/s]


In [8]:
deal_invite_info.head()

,uid,qid,qtopic
0,M401693808,Q2166419046,T456
1,M401693808,Q2166419046,T112
2,M401693808,Q2166419046,T9566
3,M401693808,Q2166419046,T5310
4,M3392373099,Q1550017551,T2


In [9]:
deal_invite_info.shape

(26315557, 3)

In [10]:
deal_invite_info=pd.merge(deal_invite_info,invite_info[['uid','qid','target']],how='left',on=['uid','qid'])
deal_invite_info=pd.merge(deal_invite_info,invite_info[['uid','qid','qtopic_len']],how='left',on=['uid','qid'])

In [11]:
member_info.head()

,uid,sex,look_freq,a,b,c,d,e,A,B,C,D,E,salt
0,M1934753188,male,monthly,0,1,0,1,0,MD470265,BR470265,PV929066,CT929066,PF470265,764
1,M595924114,male,daily,0,0,0,1,1,MD195122,BR596936,PV002320,CT840234,PF470265,671
2,M1473482940,female,weekly,0,1,0,1,0,MD116493,BR641329,PV170953,CT470265,PF470265,454
3,M578477092,male,daily,1,1,0,1,0,MD889589,BR803759,PV545833,CT545833,PF470265,588
4,M1088794709,male,weekly,0,1,0,0,0,MD825760,BR641329,PV071037,CT470265,PF470265,361


In [12]:
cat_col=['sex','look_freq','A','B','C','D','E']
import pickle
from tqdm import tqdm
from sklearn.preprocessing import LabelEncoder
for feat in tqdm(cat_col):
    with open('meta_'+str(feat)+'.pickle', 'rb') as f:
        lbl = pickle.load(f)
    member_info[feat]=lbl.transform(member_info[feat].astype('str'))

100%|██████████| 7/7 [00:05<00:00,  1.58it/s]


In [13]:
deal_invite_info=pd.merge(deal_invite_info,member_info,how='left',on='uid')

In [14]:
with open('meta_qtopic.pickle', 'rb') as f:
        lbl = pickle.load(f)
deal_invite_info['qtopic']=lbl.transform(deal_invite_info['qtopic'].astype('str'))

In [15]:
##########################################################################################

In [16]:
cat_col=['qtopic','sex','look_freq','a','b','c','d','e','A','B','C','D','E']

In [17]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold,StratifiedKFold,GroupKFold,RepeatedKFold
import warnings
import gc

In [18]:
drop_features = ['qid', 'uid','target']
feats = [f for f in deal_invite_info.columns if f not in drop_features]

n_splits= 3
folds = GroupKFold(n_splits=n_splits)

In [19]:
deal_invite_info['id']=deal_invite_info['qid']+deal_invite_info['uid']

In [20]:
len(deal_invite_info)

26398729

In [21]:
import numpy as np
feature_importance_df = pd.DataFrame()
stack_train = np.zeros((len(deal_invite_info), 1))
#stack_test = np.zeros((len(test), 1))

In [22]:
params = {
        "objective" : "binary", 
        "boosting" : "gbdt", 
        "metric" : "auc",  
        "max_depth": 7, 
        "num_leaves" : 31, 
        "max_bin" : 255, 
        "learning_rate" : 0.1, 
        "subsample" : 0.8,
        "colsample_bytree" : 0.8, 
        "verbosity": -1,
        # "num_threads" : 15,
}

In [50]:
target=deal_invite_info['target']

In [23]:
deal_invite_info.head()

,uid,qid,qtopic,target,qtopic_len,sex,look_freq,a,b,c,d,e,A,B,C,D,E,salt,id
0,M401693808,Q2166419046,37444,0,4,2,4,0,1,0,0,0,2113,190,261,927,1,297,Q2166419046M401693808
1,M401693808,Q2166419046,1331,0,4,2,4,0,1,0,0,0,2113,190,261,927,1,297,Q2166419046M401693808
2,M401693808,Q2166419046,65863,0,4,2,4,0,1,0,0,0,2113,190,261,927,1,297,Q2166419046M401693808
3,M401693808,Q2166419046,43860,0,4,2,4,0,1,0,0,0,2113,190,261,927,1,297,Q2166419046M401693808
4,M3392373099,Q1550017551,11023,0,2,2,1,1,0,0,0,0,1190,130,266,758,0,308,Q1550017551M3392373099


In [55]:
for n_fold, (trn_idx, val_idx) in enumerate(folds.split(stack_train, deal_invite_info['target'],groups=deal_invite_info['id'])):
    print("fold n°{}".format(n_fold))
    trn_data = lgb.Dataset(deal_invite_info[feats].iloc[trn_idx],
                           label=target[trn_idx],
                           categorical_feature=cat_col)
    val_data = lgb.Dataset(deal_invite_info[feats].iloc[val_idx],
                           label=target.iloc[val_idx],
                           categorical_feature=cat_col)
    num_round = 2500
    clf = lgb.train(params,
                    trn_data,
                    num_round,
                    valid_sets=[trn_data, val_data],
                    verbose_eval=100,
                    early_stopping_rounds=30)
    save_name = "topic_meta_lgb_%d" % n_fold
    clf.save_model(save_name + '_booster.txt')
    stack_train[val_idx]=clf.predict(deal_invite_info[feats].iloc[val_idx], num_iteration=clf.best_iteration).reshape(-1,1)
    #stack_test+= clf.predict(test[feats], num_iteration=clf.best_iteration).reshape(-1,1)#

fold n°0


/root/anaconda3/lib/python3.7/site-packages/lightgbm/basic.py:1291: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')


Training until validation scores don't improve for 30 rounds
[100]	training's auc: 0.677794	valid_1's auc: 0.670684
[200]	training's auc: 0.688088	valid_1's auc: 0.678705
[300]	training's auc: 0.693959	valid_1's auc: 0.682141
[400]	training's auc: 0.697869	valid_1's auc: 0.684283
[500]	training's auc: 0.702059	valid_1's auc: 0.686402
[600]	training's auc: 0.705203	valid_1's auc: 0.687755
[700]	training's auc: 0.708185	valid_1's auc: 0.688903
[800]	training's auc: 0.710725	valid_1's auc: 0.689716
[900]	training's auc: 0.712759	valid_1's auc: 0.690192
[1000]	training's auc: 0.714594	valid_1's auc: 0.690606
[1100]	training's auc: 0.716084	valid_1's auc: 0.69096
[1200]	training's auc: 0.717627	valid_1's auc: 0.691317
[1300]	training's auc: 0.719141	valid_1's auc: 0.691715
[1400]	training's auc: 0.720788	valid_1's auc: 0.69221
[1500]	training's auc: 0.722279	valid_1's auc: 0.692587
[1600]	training's auc: 0.723813	valid_1's auc: 0.693106
[1700]	training's auc: 0.725267	valid_1's auc: 0.69336

/root/anaconda3/lib/python3.7/site-packages/lightgbm/basic.py:1291: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')


Training until validation scores don't improve for 30 rounds
[100]	training's auc: 0.678292	valid_1's auc: 0.672283
[200]	training's auc: 0.688763	valid_1's auc: 0.680098
[300]	training's auc: 0.694641	valid_1's auc: 0.683555
[400]	training's auc: 0.698957	valid_1's auc: 0.686012
[500]	training's auc: 0.702687	valid_1's auc: 0.687783
[600]	training's auc: 0.705636	valid_1's auc: 0.688888
[700]	training's auc: 0.70834	valid_1's auc: 0.689753
[800]	training's auc: 0.711042	valid_1's auc: 0.690597
[900]	training's auc: 0.713092	valid_1's auc: 0.691207
[1000]	training's auc: 0.714872	valid_1's auc: 0.691652
[1100]	training's auc: 0.716523	valid_1's auc: 0.692091
[1200]	training's auc: 0.71813	valid_1's auc: 0.692449
[1300]	training's auc: 0.719891	valid_1's auc: 0.692932
[1400]	training's auc: 0.721327	valid_1's auc: 0.693199
[1500]	training's auc: 0.722842	valid_1's auc: 0.693623
[1600]	training's auc: 0.724372	valid_1's auc: 0.693894
Early stopping, best iteration is:
[1642]	training's a

/root/anaconda3/lib/python3.7/site-packages/lightgbm/basic.py:1291: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')


Training until validation scores don't improve for 30 rounds
[100]	training's auc: 0.678894	valid_1's auc: 0.670488
[200]	training's auc: 0.68918	valid_1's auc: 0.678479
[300]	training's auc: 0.694944	valid_1's auc: 0.681892
[400]	training's auc: 0.699016	valid_1's auc: 0.684259
[500]	training's auc: 0.702967	valid_1's auc: 0.686232
[600]	training's auc: 0.706152	valid_1's auc: 0.6876
[700]	training's auc: 0.709046	valid_1's auc: 0.688728
[800]	training's auc: 0.711432	valid_1's auc: 0.68936
[900]	training's auc: 0.713552	valid_1's auc: 0.689995
[1000]	training's auc: 0.715368	valid_1's auc: 0.690545
[1100]	training's auc: 0.71708	valid_1's auc: 0.690952
[1200]	training's auc: 0.71853	valid_1's auc: 0.691261
[1300]	training's auc: 0.71998	valid_1's auc: 0.691613
[1400]	training's auc: 0.72138	valid_1's auc: 0.691912
[1500]	training's auc: 0.722691	valid_1's auc: 0.692223
[1600]	training's auc: 0.72408	valid_1's auc: 0.692518
[1700]	training's auc: 0.725585	valid_1's auc: 0.692981
[1800

In [24]:
invite_info_evaluate=pd.read_csv('invite_info_evaluate_2_0926.txt',sep='\s+',names=['qid','uid','time'])
invite_info_evaluate=pd.merge(invite_info_evaluate,ques_info,on='qid',how='left')
invite_info_evaluate=invite_info_evaluate.dropna(subset=['qtopic'])
invite_info_evaluate['qtopic'] = invite_info_evaluate['qtopic'].map(lambda x:x.split(','))
invite_info_evaluate['qtopic_len'] = invite_info_evaluate['qtopic'].map(lambda x:len(x))
deal_invite_info_evaluate = flatten_topic(invite_info_evaluate)
deal_invite_info_evaluate=pd.merge(deal_invite_info_evaluate,invite_info_evaluate[['uid','qid','qtopic_len']],how='left',on=['uid','qid'])
deal_invite_info_evaluate=pd.merge(deal_invite_info_evaluate,member_info,how='left',on='uid')

100%|██████████| 1141718/1141718 [00:20<00:00, 57060.00it/s]


In [25]:
with open('meta_qtopic.pickle', 'rb') as f:
    lbl = pickle.load(f)
deal_invite_info_evaluate['qtopic']=lbl.transform(deal_invite_info_evaluate['qtopic'].astype('str'))

In [26]:
stack_train2 = np.zeros((len(deal_invite_info), 1))
stack_test = np.zeros((len(deal_invite_info_evaluate), 1))
for n_fold, (trn_idx, val_idx) in enumerate(folds.split(stack_train2, deal_invite_info['target'],groups=deal_invite_info['id'])):
    print("fold n°{}".format(n_fold))
    import lightgbm as lgb
    lgb = lgb.Booster(model_file = 'topic_meta_lgb_%d_booster.txt' % n_fold)
    stack_train2[val_idx]=lgb.predict(deal_invite_info[feats].iloc[val_idx]).reshape(-1,1)
    stack_test+= lgb.predict(deal_invite_info_evaluate[feats]).reshape(-1,1)

fold n°0
fold n°1
fold n°2


In [27]:
stack_test=stack_test/3
stack_train2=pd.DataFrame(stack_train2)
stack_test=pd.DataFrame(stack_test)
stack_train2['uid']=deal_invite_info['uid']
stack_train2['qid']=deal_invite_info['qid']
stack_test['uid']=deal_invite_info_evaluate['uid']
stack_test['qid']=deal_invite_info_evaluate['qid']

In [28]:
train_fea=stack_train2.groupby(['uid','qid']).agg(['mean','std','min','max','median']).reset_index()
test_fea=stack_test.groupby(['uid','qid']).agg(['mean','std','min','max','median']).reset_index()

In [29]:
train_fea.columns=['uid','qid','topic_meta_mean','topic_meta_std','topic_meta_min','topic_meta_max','topic_meta_median']
test_fea.columns=['uid','qid','topic_meta_mean','topic_meta_std','topic_meta_min','topic_meta_max','topic_meta_median']

In [30]:
x=pd.concat([train_fea,test_fea])

In [31]:
x=x.drop_duplicates(subset=['uid','qid'])

In [46]:
x.to_hdf('feature_topic_meta_new.h5', key='data')

In [76]:
time_end=time.time()
print('totally cost',time_end-time_start)

totally cost 30450.72513628006


In [42]:
x.to_csv('feature_topic_meta.csv',index=False)

In [43]:
time_end=time.time()
print('totally cost',time_end-time_start)

totally cost 54874.5562312603
